## Calculate Phi vertical mean for a particular layer and LME

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
a1,a2,b1,b2 = [240,380,125,320]

In [3]:
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
dset = xr.open_dataset(filename)
dz = dset['dz']/1e2 # m
darea = dset['TAREA']/1e10 #km*km
dvol = dz*darea

In [4]:
KMT = dset['KMT']
nlat = dset.dims['nlat']
nlon = dset.dims['nlon']
for xx in range(nlat):
    for yy in range(nlon):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            dvol[kk:, xx, yy] = float("nan")
dvol = dvol.isel(z_t=range(0, 35), nlat=range(a1, a2), nlon=range(b1, b2))

In [5]:
filename = f'{path}/DPLE_driftcorrected_Phi_mean_dimension_reduced_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)
dsf = xr.Dataset()
for ii in range(3):
    Phi_i = ds[f'Phi_{ii}']*dvol
    # select a vertical layer
    dsf[f'Phi_{ii}_0-200m'] = Phi_i.isel(z_t=range(0, 20, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(0, 20, 1)).sum(dim='z_t'))
    dsf[f'Phi_{ii}_200-600m'] = Phi_i.isel(z_t=range(20, 35, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(20, 35, 1)).sum(dim='z_t'))

In [6]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:         (nlat: 140, nlon: 195, L: 122)
Coordinates:
    ULONG           (nlat, nlon) float64 101.7 102.9 104.0 ... 318.9 319.5 320.0
    ULAT            (nlat, nlon) float64 15.13 15.13 15.14 ... 70.64 70.63 70.63
    Y               int64 1954
  * L               (L) int64 1 2 3 4 5 6 7 8 ... 116 117 118 119 120 121 122
    TLAT            (nlat, nlon) float64 14.96 14.96 14.97 ... 70.43 70.42 70.42
    TLONG           (nlat, nlon) float64 101.2 102.3 103.4 ... 318.6 319.2 319.7
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_0_0-200m    (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_0_200-600m  (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_1_0-200m    (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_1_200-600m  (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_2_0-200m    (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan
    Phi_2_200-600m  (L, nlat, nlon) float64 nan nan nan nan ... nan nan nan nan

In [7]:
%time
#dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_mean_dimension_reduced_vertical_mean_ens_mean_monthly.nc', mode='w')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


## calculate LME-averaged Phi 

In [3]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE_driftcorrected_Phi_mean_dimension_reduced_vertical_mean_ens_mean_monthly.nc'
dsf = xr.open_dataset(filename)

In [4]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask']

In [5]:
#lme = range(np.int(lme_mask.min()), np.int(lme_mask.max()), 1)
lme = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 65]
nlead = dsf.dims['L']

In [6]:
boxPhi = xr.Dataset()
for ii in range(3):
    print(ii)
    phi_box1 = np.ma.zeros([len(lme), nlead])
    phi_box1.mask = True
    phi_box2 = phi_box1.copy()
    for box, hh in zip(lme, range(len(lme))):
        latloc, lonloc = np.where(lme_mask == box)
        latloc = list(np.array(latloc) - a1)
        lonloc = list(np.array(lonloc) - b1)
        ar1 = np.ma.zeros([len(latloc),nlead]);ar1.mask=True
        ar2 = ar1.copy()
        for bb in range(len(latloc)):
            ar1[bb] = dsf[f'Phi_{ii}_0-200m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
            ar2[bb] = dsf[f'Phi_{ii}_200-600m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
        phi_box1[hh] = np.nanmean(ar1,axis=0)
        phi_box2[hh] = np.nanmean(ar2,axis=0)
        #
        #phi_box1[hh] = dsf[f'Phi_{ii}_0-200m'].isel(nlat=latloc, nlon=lonloc).mean(dim='nlat').mean(dim='nlon')
        #phi_box2[hh] = dsf[f'Phi_{ii}_200-600m'].isel(nlat=latloc, nlon=lonloc).mean(dim='nlat').mean(dim='nlon')
    boxPhi[f'Phi_{ii}_0-200m'] = xr.DataArray(data=phi_box1, coords=dict(lme=('lme', lme), L=('L', dsf.coords['L'].values)), dims=('lme','L'))
    boxPhi[f'Phi_{ii}_200-600m'] = xr.DataArray(data=phi_box2, coords=dict(lme=('lme', lme), L=('L', dsf.coords['L'].values)), dims=('lme','L'))

0
1
2


In [7]:
boxPhi.load()
dout = f'/glade/scratch/{USER}/DPLE-results'
#boxPhi.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_LME_ens_mean_monthly.nc', mode='w')
boxPhi.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_mean_dimension_reduced_LME_ens_mean_monthly.nc', mode='w')

## Calculate Yearly

In [2]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """

    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)

In [3]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE_driftcorrected_Phi_mean_dimension_reduced_vertical_mean_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)
dsf = xr.Dataset()
for ii in range(3):
    # select a vertical layer
    for layer in ['0-200m','200-600m']:
        dsf[f'Phi_{ii}_{layer}'] = xr_reshape(ds[f'Phi_{ii}_{layer}'][2:,:,:], 'L', ['year', 'mon'], [np.arange(10), np.arange(12)]).mean(dim='mon')

In [4]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:         (nlat: 140, nlon: 195, year: 10)
Coordinates:
    ULONG           (nlat, nlon) float64 101.7 102.9 104.0 ... 318.9 319.5 320.0
    ULAT            (nlat, nlon) float64 15.13 15.13 15.14 ... 70.64 70.63 70.63
    Y               int64 1954
    TLAT            (nlat, nlon) float64 14.96 14.96 14.97 ... 70.43 70.42 70.42
    TLONG           (nlat, nlon) float64 101.2 102.3 103.4 ... 318.6 319.2 319.7
  * year            (year) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_0_0-200m    (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_0_200-600m  (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1_0-200m    (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_1_200-600m  (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2_0-200m    (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan
    Phi_2_200-600m  (year, nlat, nlon) float64 nan nan nan nan ... nan nan nan

In [5]:
%%time
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_mean_dimension_reduced_vertical_mean_ens_mean_monthly_to_yearly.nc', mode='w')

CPU times: user 39.7 ms, sys: 5.81 ms, total: 45.5 ms
Wall time: 87.5 ms


## Yearly LME

In [4]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE_driftcorrected_Phi_mean_dimension_reduced_vertical_mean_ens_mean_monthly_to_yearly.nc'
dsf = xr.open_dataset(filename)

In [5]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask']

In [6]:
#lme = range(np.int(lme_mask.min()), np.int(lme_mask.max()), 1)
lme = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 65]
nlead = dsf.dims['year']

In [8]:
boxPhi = xr.Dataset()
for ii in range(3):
    print(ii)
    phi_box1 = np.ma.zeros([len(lme), nlead])
    phi_box1.mask = True
    phi_box2 = phi_box1.copy()
    for box, hh in zip(lme, range(len(lme))):
        latloc, lonloc = np.where(lme_mask == box)
        latloc = list(np.array(latloc) - a1)
        lonloc = list(np.array(lonloc) - b1)
        ar1 = np.ma.zeros([len(latloc),nlead]);ar1.mask=True
        ar2 = ar1.copy()
        for bb in range(len(latloc)):
            ar1[bb] = dsf[f'Phi_{ii}_0-200m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
            ar2[bb] = dsf[f'Phi_{ii}_200-600m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
        phi_box1[hh] = np.nanmean(ar1,axis=0)
        phi_box2[hh] = np.nanmean(ar2,axis=0)
        #
        #phi_box1[hh] = dsf[f'Phi_{ii}_0-200m'].isel(nlat=latloc, nlon=lonloc).mean(dim='nlat').mean(dim='nlon')
        #phi_box2[hh] = dsf[f'Phi_{ii}_200-600m'].isel(nlat=latloc, nlon=lonloc).mean(dim='nlat').mean(dim='nlon')
    boxPhi[f'Phi_{ii}_0-200m'] = xr.DataArray(data=phi_box1, coords=dict(lme=('lme', lme), L=('year', dsf.coords['year'].values)), dims=('lme','year'))
    boxPhi[f'Phi_{ii}_200-600m'] = xr.DataArray(data=phi_box2, coords=dict(lme=('lme', lme), L=('year', dsf.coords['year'].values)), dims=('lme','year'))

0
1
2


In [10]:
boxPhi.load()
dout = f'/glade/scratch/{USER}/DPLE-results'
#boxPhi.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_LME_ens_mean_monthly.nc', mode='w')
boxPhi.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_mean_dimension_reduced_LME_ens_mean_monthly_to_yearly.nc', mode='w')